In [1]:
import pandas as pd

In [2]:
mannKendallResult = pd.read_csv("../4-StatisticalTests/mann_kendall_results.csv")

In [3]:
splitted = pd.read_csv("splitted.csv")

In [4]:
def split_values(results):
    pop = pd.DataFrame()
    unpop = pd.DataFrame()
    for index in range(0, len(results)):

        sample = results.iloc[index]
        sample_splitted = splitted[splitted["path"] == sample["full_name"]]

        if((sample_splitted["popularity"] == "popular").bool()):
            pop = pop.append(sample, ignore_index=True)
        elif((sample_splitted["popularity"] == "unpopular").bool()):
            unpop = unpop.append(sample, ignore_index=True)
    return pop, unpop

In [5]:
pop, unpop = split_values(mannKendallResult)

In [6]:
def count_values(popularity):
    llist = list()
    for metric in popularity.columns.values:
        metrics = popularity[metric]
        if("trend" in metric and not "question" in metric and not "Readability" in metric):
            metrics = metrics.value_counts()
            llist.append(
                {
                    "metric": " ".join(metric.split(" ")[:-1]),
                    "no trend": metrics["no trend"], 
                    "increasing": metrics["increasing"], 
                    "decreasing": metrics["decreasing"]
                }
            )
    return llist

In [7]:
pd.DataFrame(count_values(pop)).to_csv("mann_kendall_popular.csv", index=False)

In [8]:
pd.DataFrame(count_values(unpop)).to_csv("mann_kendall_unpopular.csv", index=False)

Correlation

In [9]:
correlationResults = pd.read_csv("../4-StatisticalTests/correlation_results.csv")

In [10]:
pop, unpop = split_values(correlationResults)

In [11]:
def count_values(popularity):
    correlationResultSum = list()
    correlation = ""
    pvalue = ""
    for metric in popularity.columns.values:
        metrics = popularity[metric]
        if ("framework" in metric):
            continue
        if("correlation" in metric):
            correlation = metric
        elif("pvalue" in metric):
            pvalue = metric
        if (correlation and pvalue and correlation.split(" ")[:-1] == pvalue.split(" ")[:-1]):
            correlationResultSum.append(
                {
                    "metric": " ".join(metric.split(" ")[:-1]),
                    "positive correlation": len(popularity[(popularity[pvalue] < 0.05) & (popularity[correlation] > 0)]), 
                    "negative correlation": len(popularity[(popularity[pvalue] < 0.05) & (popularity[correlation] < 0)]), 
                    "inconclusive": len(popularity[pd.isna(popularity[pvalue])]) + len(popularity[popularity[pvalue] > 0.05])
                }
            )
    return correlationResultSum

In [12]:
pd.DataFrame(count_values(pop)).to_csv("correlation_popular.csv", index=False)

In [13]:
pd.DataFrame(count_values(unpop)).to_csv("correlation_unpopular.csv", index=False)